# Discriminante Quadratico Gaussiano

## Importações

In [1]:
%run utils.ipynb

## Carregando e Organizando Dados


In [2]:
#Carregando dados
dados = np.genfromtxt("iris.data", delimiter=',', skip_header=0)

#Embaralha instancias
np.random.shuffle(dados)

#Separa X e y
X = dados[:,:4]
y = dados[:,4]

#Separa o conjunto de treino (60%) e o de teste (40%)
pTreino = X[:int(dados.shape[0]*0.6)]
pTeste = X[int(dados.shape[0]*0.6):]
pTreinoAlvo = y[:int(dados.shape[0]*0.6)]
pTesteAlvo = y[int(dados.shape[0]*0.6):]


## Objeto

In [3]:
class DiscriminanteQuadraticoGaussiano():
    def __init__(self):
        self._estimator_type = "classifier"
        self.classes = {}
        self.probs = {}
        self.covs = {}
        self.means = {}
        pass

    def fit(self, X, y):
        #Separa por classe - Unique retorna um array com os valores do vetor de entrada, sem repetições
        self.classes, qtdY = np.unique(y, return_counts=True)

        #Para cada classe, criamos a matriz covariante
        for classe, atualY in zip(self.classes, qtdY):

            #Probabilidade geral de cada classe
            self.probs[classe] = (atualY/qtdY.sum())

            #Incides em que a classe ocorre
            indices = np.where(y == classe)
            #print(indices[0])
            #Definição do novo X da respectiva classe 
            x = np.array([X[index, :] for index in indices])
            x = x[0]
            x.reshape(atualY, X.shape[1])  #Reshape pra deixar no formato equivalente

            #Finalmente, definimos a matriz covariante
            self.covs[classe] = np.cov(x, rowvar=False)

            #Media de cada classe
            self.means[classe] = x.mean(axis=0)

    def predict(self, X):
        predict = np.array([])

        for x in X: 
            pcxs = np.array([])

            for classe in self.classes:
                divisor = np.exp((-1/2) * (x - self.means[classe]).T @ (np.linalg.inv(self.covs[classe])) @ (x - self.means[classe]))
                denominador = np.sqrt(np.linalg.det(self.covs[classe])) * ((2*math.pi) ** (X.shape[1]/2))
                
                pcx = divisor/denominador
                pc = self.probs[classe]

                pcxs = np.append(pcxs, pcx * pc)

            pcxMax = self.classes[np.where(pcxs == pcxs.max())]
            predict = np.append(predict, pcxMax)    
        
        return predict

## Treinamento e Predição


In [4]:
DQG = DiscriminanteQuadraticoGaussiano()
DQG.fit(pTreino, pTreinoAlvo)
predicao = DQG.predict(pTeste)
print(predicao)

[3. 1. 3. 3. 1. 1. 3. 1. 3. 3. 2. 1. 3. 3. 1. 3. 1. 1. 2. 2. 2. 3. 1. 3.
 3. 2. 1. 3. 3. 2. 3. 2. 1. 2. 3. 1. 1. 2. 3. 2. 3. 1. 2. 3. 2. 2. 3. 1.
 2. 3. 2. 2. 3. 3. 3. 2. 3. 1. 2. 3.]


## Avaliação


In [5]:
print("Porcentagem de Acerto: ", porcentagemAcerto(predicao, pTesteAlvo))

Porcentagem de Acerto:  0.9833333333333333


## Matriz de Confusão

In [22]:
matrizConfusao(predicao, pTesteAlvo, DQG)

IndexError: tuple index out of range